# Tuning Hyperparameters

__Attention__:   
    This approach can be used for some super small neural nets. Because we're basically trying to bruteforce the right combination. 
    
    There are other packages for Keras e.g. Hyperas, Hyperopt,...

## The underlying idea

We gonna wrap our models into Sklearn models so that we can use the RandomizedSearch and GridSearch functions there

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape = [9]): 
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(name= "input"))
    # model.add(keras.layers.InputLayer(input_shape = input_shape))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation = keras.activations.relu))
        
    model.add(keras.layers.Dense(1, name= "output"))
    model.compile(optimizer = keras.optimizers.SGD(learning_rate = learning_rate), loss = "mse")
    return model

In [ ]:
# Building the Wrapper around Keras Model

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

## Loading Data

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
## Validation Dataset is missing

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
x_valid, x_train = x_train[:5000], x_train[5000:]

In [ ]:
# Creating the validation dataset from the training data. We use 5000 the rest will be used for training. 

In [ ]:
y_valid, y_train = y_train[:5000], y_train[5000:]

In [ ]:
## We should scale our data because we're gonna use Gradient Descent.

In [ ]:
x_valid, x_train, x_test = x_valid/255, x_train/255, x_test/255

In [ ]:
## Using Scikit-Learn to fit, predict and hyperparamter tunning

In [110]:
keras_reg.fit(x_train, y_train, epochs = 10, validation_data = (x_valid, y_valid), callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/10
1719/1719 [==============================] - 1s 778us/step - loss: 2.4200 - val_loss: 2.0003
Epoch 2/10
1719/1719 [==============================] - 1s 742us/step - loss: 1.8157 - val_loss: 1.6706
Epoch 3/10
1719/1719 [==============================] - 1s 739us/step - loss: 1.6925 - val_loss: 1.9905
Epoch 4/10
1719/1719 [==============================] - 1s 727us/step - loss: 1.6165 - val_loss: 1.5594
Epoch 5/10
1719/1719 [==============================] - 1s 728us/step - loss: 1.5641 - val_loss: 1.6163
Epoch 6/10
1719/1719 [==============================] - 1s 732us/step - loss: 1.5126 - val_loss: 1.5113
Epoch 7/10
1719/1719 [==============================] - 1s 737us/step - loss: 1.4733 - val_loss: 1.7293
Epoch 8/10
1719/1719 [==============================] - 1s 737us/step - loss: 1.4422 - val_loss: 1.4453
Epoch 9/10
1719/1719 [==============================] - 1s 729us/step - loss: 1.4116 - val_loss: 1.4692
Epoch 10/10
1719/1719 [==============================] - 1s 735u

In [111]:
keras_reg.score(x_test, y_test)

313/313 [==============================] - 0s 554us/step - loss: 1.5150


-1.5149667263031006

In [112]:
keras_reg.predict(x_test[:3])

array([7.7330155 , 2.5430045 , 0.90628546], dtype=float32)

## Hyperparamter Tunning

In [117]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

In [118]:
params = {
    "n_hidden": [5,3,2],
    "n_neurons": np.arange(20),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [119]:
rnd_search = RandomizedSearchCV(keras_reg, params, n_iter=10, cv=3) # n_iter: number of paramter setting that are sampled, cv: Cross Validation Split

In [121]:
rnd_search.fit(x_train, y_train, epochs = 10, validation_data = (x_valid, y_valid), callbacks = [keras.callbacks.EarlyStopping(patience = 10)])

Epoch 1/10
1146/1146 [==============================] - 1s 815us/step - loss: 3.2895 - val_loss: 2.2259
Epoch 2/10
1146/1146 [==============================] - 1s 735us/step - loss: 2.1295 - val_loss: 1.9364A: 0s - loss: 2.1
Epoch 3/10
1146/1146 [==============================] - 1s 760us/step - loss: 1.9245 - val_loss: 1.8368
Epoch 4/10
1146/1146 [==============================] - 1s 727us/step - loss: 1.8131 - val_loss: 1.7491
Epoch 5/10
1146/1146 [==============================] - 1s 755us/step - loss: 1.7398 - val_loss: 1.6954
Epoch 6/10
1146/1146 [==============================] - 1s 734us/step - loss: 1.6868 - val_loss: 1.6562
Epoch 7/10
1146/1146 [==============================] - 1s 724us/step - loss: 1.6394 - val_loss: 1.7180
Epoch 8/10
1146/1146 [==============================] - 1s 754us/step - loss: 1.6075 - val_loss: 1.6276
Epoch 9/10
1146/1146 [==============================] - 1s 752us/step - loss: 1.5840 - val_loss: 1.5845
Epoch 10/10
573/573 [==========================

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000027D60C3AE10>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000027D46EABFD0>,
                                        'n_hidden': [5, 3, 2],
                                        'n_neurons': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])})